# Model regresji logistycznej dla klasyfikacji przeżycia pasażerów Titanica

## Wprowadzenie

W tym projekcie budujemy model regresji logistycznej do klasyfikacji binarnej (*przeżył – nie przeżył*) pasażerów Titanica. Regresja logistyczna jest standardową metodą dla problemów, w których zmienna zależna przyjmuje dwie wartości (np. "przeżył" vs. "nie przeżył") [statystyka.eu](https://statystyka.eu).

W danych Titanic udokumentowano, że takie czynniki jak płeć czy klasa podróży silnie wpływały na szanse przeżycia – przykładowo **~73% kobiet przeżyło**, podczas gdy **tylko ~19% mężczyzn wróciło żywych** [shiftcomm.com](https://shiftcomm.com). Podobnie **61.9% pasażerów I klasy przeżyło**, ale tylko **25.5% pasażerów III klasy** [shiftcomm.com](https://shiftcomm.com).

Regresja logistyczna pozwala uwzględnić takie czynniki (*zmienne cechowe*) i oszacować ich wpływ na prawdopodobieństwo przeżycia.

## Wczytanie i wstępna analiza danych

Na początek wczytujemy dane o pasażerach i przyglądamy się ich strukturze. Zakładamy, że dane są dostępne w pliku `tested.csv` (analogiczny do zbioru Kaggle). Używamy biblioteki `pandas` do ich analizy.

In [15]:
import pandas as pd

# 1. Wczytanie danych
df = pd.read_csv('tested.csv') 
print(df.shape)            # liczba wierszy i kolumn
print(df.isnull().sum())   # brakujące wartości w poszczególnych kolumnach

(418, 12)
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


Komentarz: Z otrzymanych wyników widzimy, że mamy 418 rekordów. Część kolumn (`Age`, `Fare`, `Cabin`) ma brakujące wartości. W szczególności aż 86 braków w wieku i 327 w kabinie (Cabin), dlatego `Cabin` wyrzucamy z analizy ze względu na zbyt duży procent braków.


## Przygotowanie danych

Musimy teraz przygotować dane do modelu: usunąć nieistotne cechy, uzupełnić brakujące wartości i zakodować zmienne kategoryczne.

- **Usuwamy kolumny**, które nie są przydatne (np. `Name`, `Ticket`, `PassengerId` oraz `Cabin`, który zawiera głównie brakujące dane).
- **Wypełniamy braki**: 
  - Zastępujemy brakujący `Age` medianą kolumny.
  - Zastępujemy brakujący `Fare` medianą (inna strategia to średnia lub regresja; tutaj prosta mediana).
- **Kodujemy zmienne**:
  - `Sex` kodujemy binarnie (np. `male=0`, `female=1`).
  - `Embarked` (port zaokrętowania) kodujemy metodą one-hot (tworzymy wskaźniki np. `Embarked_Q`, `Embarked_S`, pomijając jedną kategorię jako bazową).

In [16]:
from sklearn.model_selection import train_test_split

# 2. Przygotowanie danych: usunięcie niepotrzebnych cech i uzupełnienie braków
df = df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])  # niepotrzebne dla modelu
df['Age'].fillna(df['Age'].median(), inplace=True)    # uzupełnienie braków medianą
df['Fare'].fillna(df['Fare'].median(), inplace=True)

# 3. Kodowanie zmiennych kategorycznych
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})   # płeć jako 0/1
df = pd.get_dummies(df, columns=['Embarked'], drop_first=True)  # one-hot dla kolumny Embarked

print(df.isnull().sum())   # brakujące wartości po przetworzeniu
print(df.columns)      

Survived      0
Pclass        0
Sex           0
Age           0
SibSp         0
Parch         0
Fare          0
Embarked_Q    0
Embarked_S    0
dtype: int64
Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked_Q', 'Embarked_S'],
      dtype='object')


C:\Users\krzys\AppData\Local\Temp\ipykernel_17304\873929125.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].median(), inplace=True)    # uzupełnienie braków medianą
C:\Users\krzys\AppData\Local\Temp\ipykernel_17304\873929125.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values a

Komentarz: Po tych krokach nie ma braków danych (`Age` i `Fare` uzupełnione). Mamy tylko cechy numeryczne: `Pclass`, `Sex`, `Age`, `SibSp`, `Parch`, `Fare`, oraz nowe wskaźniki z kodowania `Embarked_Q`, `Embarked_S`. Zmienna docelowa `Survived` pozostaje bez zmian (0/1).

## Podział na zbiory treningowy i testowy
Następnie dzielimy dane na zbiór treningowy i testowy, np. w proporcji 2/3 : 1/3. Zbiór treningowy wykorzystamy do nauczenia modelu, a testowy – do oceny jakości prognoz na nowych danych.

In [17]:
# 4. Przygotowanie zbiorów treningowego i testowego
X = df.drop('Survived', axis=1)
y = df['Survived']

# dzielimy dane losowo: 2/3 na trening, 1/3 na test
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.33, 
                                                    random_state=42)


Komentarz: Ustalając `random_state=42`, zapewniamy powtarzalność podziału (można użyć dowolnej liczby całkowitej). Po podziale sprawdzamy rozmiary: powinny być proporcjonalne (np. ~280 obserwacji w zbiorze treningowym i ~138 w testowym).

## Trening modelu regresji logistycznej

Tworzymy obiekt modelu regresji logistycznej i trenujemy go na zbiorze treningowym. Zwiększamy parametr `max_iter`, aby zapewnić zbieżność algorytmu (domyślnie 100; nasz zestaw jest niewielki, więc 1000 wystarczy).

In [18]:
from sklearn.linear_model import LogisticRegression

# 5. Trenowanie modelu regresji logistycznej
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

Komentarz: Model z metodyki **scikit-learn** implementuje **regresję logistyczną** (regularizowaną wersję), która minimalizuje funkcję **logistycznego błędu (log-loss)** i szacuje współczynniki przy zmiennych.  

Regresja logistyczna przekształca **kombinację liniową cech** (`X_train`) w **prawdopodobieństwo przeżycia** przez funkcję **logistyczną (sigmoidalną)**, a następnie decyduje o klasie (_przeżył/nie przeżył_) na podstawie progu `0.5`.

Źródło: [statystyka.eu](https://statystyka.eu)


## Ocena modelu

Po wytrenowaniu modelu sprawdzamy jego skuteczność na zbiorze testowym. Wyliczamy **dokładność (accuracy)**, **macierz pomyłek** oraz inne metryki (np. precision, recall) dla każdej klasy. Accuracy to odsetek poprawnych klasyfikacji. Macierz pomyłek pokazuje liczbę prawdziwie pozytywnych (TP), fałszywie pozytywnych (FP), itd.


In [19]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# 6. Predykcja i ocena na zbiorze testowym
y_pred = model.predict(X_test)

acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print(f"Dokładność (accuracy): {acc:.2f}")
print("Macierz pomyłek:")
print(cm)
print("\nRaport klasyfikacji (precision, recall, f1 dla każdej klasy):")
print(classification_report(y_test, y_pred))


Dokładność (accuracy): 1.00
Macierz pomyłek:
[[92  0]
 [ 0 46]]

Raport klasyfikacji (precision, recall, f1 dla każdej klasy):
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        92
           1       1.00      1.00      1.00        46

    accuracy                           1.00       138
   macro avg       1.00      1.00      1.00       138
weighted avg       1.00      1.00      1.00       138



Komentarz: Otrzymane wartości wskażą skuteczność modelu. Dokładność to 100% (1.00). Macierz pomyłek ma postać:
```
[[TN  FP]
 [FN  TP]]
```
gdzie TN to liczba poprawnie sklasyfikowanych przykładów negatywnych (np. "nie przeżył" przewidziany jako "nie przeżył"), FP – liczba fałszywych alarmów ("nie przeżył" błędnie przewidziany jako "przeżył"), itd.

W naszym przykładzie (ze zbiorem danych, w którym wszystkie kobiety przeżyły, a wszyscy mężczyźni zginęli) model osiągnął idealną skuteczność 100% na zbiorze testowym (brak błędnych klasyfikacji).

Interpretacja: model poprawnie przewidział 92 osoby, które zginęły, oraz 46 osób, które przeżyły. Nie ma błędów fałszywie pozytywnych ani negatywnych (co daje accuracy=1.00). Jednak należy zauważyć, że jest to efekt bardzo silnego wzmocnienia w danych – w rzeczywistości Titanic miał trochę inne proporcje przeżycia (np. ~73% kobiet i ~19% mężczyzn) [shiftcomm.com](https://shiftcomm.com)

Metryki dodatkowe (**precision**, **recall**, **f1**) również są równe 1.00 dla obu klas, co oznacza idealną klasyfikację. W praktyce jednak tego typu wynik sugeruje, że model może być nadmiernie dopasowany do specyfiki danych albo że dane są bardzo jednorodne. Z jednej strony regresja logistyczna jest prostym i efektywnym modelem (szczególnie gdy spełnione są jej założenia: binarna zmienna docelowa, brak silnej korelacji między cechami, liniowy związek log-odds ze zmiennymi niezależnymi) [predictivesolutions.pl](https://predictivesolutions.pl). Z drugiej strony, nasz zestaw danych ma specyficzny rozkład (skrajnie różne wyniki dla obu płci), więc model "zapamiętał" prostą regułę (kobieta = przeżyła, mężczyzna = nie przeżył).

